# Ground State

The hubbard hamiltonian for the non-interacting fermionic system is as follows:

#### $\hat{H} = -t\sum\limits_{\langle{r,r'}\rangle\sigma}\hat{c}^{\dagger}_{r,\sigma}\hat{c}_{r',\sigma} + \frac{U}{2}\sum\limits_{r}\sum\limits_{\sigma\neq\sigma'}\hat{n}_{\sigma}(r)\hat{n}_{\sigma'}(r) \tag{1}$

where $r$ denotes a lattice coordinate and $\sigma$ denotes the "flavor" of a particle. In ferminionic systems, one such "flavor" is a particle's spin (spin-up $\uparrow$ and spin-down $\downarrow$). We seek to implement a generalization of the Hubbard hamiltonian where a user can arbitrarily construct an $d$-dimensional lattice with $N$ particles and $F$ flavors.

In [5]:
import numpy as np
import matplotlib.pyplot as plt

We intend to construct the lattice in the single-particle Hilbert space:

$\langle{r,\sigma}|\hat{H}|{r',\sigma'}\rangle \tag{2}$

We begin by defining the parameters of the lattice system. This allows the user to choose and tune the dimensionality, specify each dimension's length, the total number of particles in the system, and the total number of particles existing in a state of a certain flavor.

In [4]:
dimension = 2
d_array = np.empty(dimension,dtype=np.int64)
for d in range(dimension):
    d_array = int(input(f"How many sites in direction {d}?"))

How many sites in direction 0?1
How many sites in direction 1?2
